# 正态性检验方法
**正态性检验（Normality Test）**是统计学中用于判断数据是否服从正态分布（高斯分布）的重要方法。正态分布是许多统计模型（如t检验、ANOVA、线性回归等）的基础假设，因此验证数据的正态性至关重要。

正态性检验方法主要有**Shapiro-Wilk检验**、**Kolmogorov-Smirnov检验 (K-S检验)**、**Anderson-Darling检验**

| 检验方法               | Shapiro-Wilk检验                          | Kolmogorov-Smirnov检验 (K-S检验)           | Anderson-Darling检验                     |
|------------------------|------------------------------------------|------------------------------------------|------------------------------------------|
| **适用场景**           | 小样本（n ≤ 50）效果更好                  | 大样本，或已知参考分布时                  | 中小样本，尤其关注尾部拟合               |
| **原假设 (H₀)**        | 数据来自正态分布                          | 数据与指定分布（如正态分布）一致           | 数据来自特定分布（如正态分布）            |
| **统计量特点**         | 基于样本数据的顺序统计量                  | 基于经验分布函数与理论分布的最大差异       | 加权平方差异（对尾部更敏感）              |
| **优点**               | 小样本功效高                              | 适用于任何分布比较，无需参数估计           | 对尾部异常敏感，适合极端值检测           |
| **缺点**               | 大样本时可能过于敏感（易拒绝）             | 对分布参数未知时功效较低（需Lilliefors修正） | 计算较复杂                               |
| **参数估计需求**       | 不需要预先估计分布参数                     | 若参数未知需估计（μ,σ），需用Lilliefors修正 | 通常需要估计分布参数                      |
| **适用数据类型**       | 连续数据                                  | 连续或离散数据                            | 连续数据                                 |
| **样本量限制**         | 3 ≤ n ≤ 5000（常见实现）                  | 无严格限制，但大样本更稳定                | 无严格限制                               |

# 数值法（统计检验法）

## Shapiro-Wilk检验

### 基本原理
**Shapiro-Wilk检验**（1965年由Samuel Shapiro和Martin Wilk提出）是一种基于顺序统计量的正态性检验方法。其核心思想是：
- 如果样本数据来自正态分布，那么数据的排序值（从小到大排列）应与理论正态分布的分位数呈高度线性相关。
- 检验通过计算一个统计量$W$（称为Shapiro-Wilk统计量）来量化这种相关性：
    - $W$值越接近1，数据越可能服从正态分布；
    - $W$值显著小于1，则拒绝正态性假设。

### 适用场景
- **小样本**（n ≤ 50）：检验功效（Power）最高，推荐优先使用。
- **中等样本**（50 < n ≤ 2000）：仍可使用，但敏感性可能降低。
- **大样本**（n > 2000）：部分软件（如R）可能无法计算，且易因微小偏离拒绝原假设（过于严格）。
- **不适用场景**： 
    - 数据含极端异常值（检验对异常值敏感）。
    - 样本量极大（如n > 5000）。

### 零假设与备择假设
- 零假设（$H_0$）：样本数据来自正态分布
- 备择假设（$H_1$）：样本数据不来自正态分布

### 检验统计量
检验统计量$W$的计算公式为：
$$W = \frac{\left( \sum_{i=1}^n a_i x_{(i)} \right)^2}{\sum_{i=1}^n (x_i - \bar{x})^2}$$
其中：
- $x_{(i)}$：第$i$个顺序统计量（排序后的数据）。
- $a_i$：理论正态分布分位数的系数（由协方差矩阵计算，查Shapiro-Wilk系数表获得）。
分母为样本方差的无偏估计。

### 检验步骤
- **数据排序**：将样本数据按升序排列，$得到 x_{(1)} \leq x_{(2)} \leq \ldots \leq x_{(n)}$。
- **计算理论分位数**： 
    - 生成标准正态分布的分位数 $m_i = \Phi^{-1}\left(\frac{i - 0.375}{n + 0.25}\right)$，其中$Φ⁻¹$为标准正态逆CDF。
- **计算系数$a_i$**： 
    - 基于理论分位数$m_i$和协方差矩阵，通过查表或算法获得。
- **计算W统计量**： 
    - 按公式计算分子（线性组合的平方）和分母（样本方差）。
- **假设检验**： 
    - 比较$W$值与临界值（或计算$p$值），若$p < α$则拒绝H₀。

### Shapiro-Wilk检验实际案例

#### 案例背景
某互联网公司希望测试新版推荐算法（B组）相比旧版（A组）是否能显著提升用户点击率（CTR）。实验设计如下：
- A组（对照组）：旧版推荐算法，用户点击率数据。
- B组（实验组）：新版推荐算法，用户点击率数据。
- 样本量：每组各30名用户（小样本）。
- 目标：验证两组数据是否服从正态分布，以决定后续所使用假设检验类型。

In [2]:
import numpy as np

# A组（旧版算法）
ctr_A = np.array([2.1, 1.8, 2.5, 2.0, 1.9, 2.2, 2.4, 1.7, 2.3, 2.6, 
                  1.6, 2.7, 2.8, 1.5, 2.9, 3.0, 1.4, 3.1, 3.2, 1.3, 
                  2.1, 1.8, 2.5, 2.0, 1.9, 2.2, 2.4, 1.7, 2.3, 2.6])

# B组（新版算法）
ctr_B = np.array([2.5, 2.8, 3.0, 2.7, 2.9, 3.1, 3.2, 2.6, 3.3, 3.5, 
                  2.4, 3.6, 3.7, 2.3, 3.8, 3.9, 2.2, 4.0, 4.1, 2.1, 
                  2.5, 2.8, 3.0, 2.7, 2.9, 3.1, 3.2, 2.6, 3.3, 3.5])

#### 确定假设
- 零假设（$H_0$）：样本数据来自正态分布
- 备择假设（$H_1$）：样本数据不来自正态分布

#### `stats.shapiro()`
- 输入参数
   - `x`: 输入待检验的数据（一维或多维数组）
   - `axis`: 对多维数组指定计算轴（如axis=0按列检验，axis=1按行检验）。默认值为None，会将数组展平为一维后检验
   - `nan_policy`: 如何处理 NaN 值：
        - `propagate`：结果为 NaN
        - `omit`：忽略 NaN
        - `raise`：抛出错误
- 返回值
    - `W`: Shapiro-Wilk统计量，范围[0, 1]，越接近1越可能正态
    - `p`: p值

#### `stats.shapiro`代码实现

In [6]:
from scipy import stats

# A组检验
W_A, p_A = stats.shapiro(ctr_A)
print(f"A组 - Shapiro-Wilk检验: W = {W_A:.4f}, p = {p_A:.4f}")
if p_A > 0.05:
    print("A组数据服从正态分布（p>0.05）")
else:
    print("A组数据不服从正态分布（p≤0.05）")
    
# B组检验
W_B, p_B = stats.shapiro(ctr_B)
print(f"B组 - Shapiro-Wilk检验: W = {W_B:.4f}, p = {p_B:.4f}")
if p_B > 0.05:
    print("B组数据服从正态分布（p>0.05）")
else:
    print("B组数据不服从正态分布（p≤0.05）")

A组 - Shapiro-Wilk检验: W = 0.9817, p = 0.8686
A组数据服从正态分布（p>0.05）
B组 - Shapiro-Wilk检验: W = 0.9749, p = 0.6805
B组数据服从正态分布（p>0.05）
